In [1]:
import numpy as np
import pickle

with open("txt_data", "rb") as fp:   
    data = pickle.load(fp)
with open("txt_labels", "rb") as fp:
    labels = pickle.load(fp)

print(len(data), len(labels))

31702 31702


In [2]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size = 0.30, shuffle=True, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, shuffle=True, random_state = 33)

print('split done.')

split done.


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500, use_idf=True, stop_words='english')
vectorizer.fit(data)
x_train = vectorizer.transform(x_train).toarray()
x_test = vectorizer.transform(x_test).toarray()
x_valid = vectorizer.transform(x_valid).toarray()
print('> tfidVectorization done.')

> tfidVectorization done.


In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from sklearn.svm import SVC

params = {
    'kernel':['rbf'],
}
clf = GridSearchCV(SVC(), param_grid=params, cv=2, n_jobs=-1, verbose=1)
clf.fit(x_train, y_train)

clf = clf.best_estimator_
print()
print('Best params: ',clf)

y_pred = clf.predict(x_test)

print()
dt = pd.DataFrame(confusion_matrix(y_test, y_pred))
display(dt)
print(classification_report(y_test, y_pred, zero_division=0))

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.3min finished



Best params:  SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)



,0,1,2,3,4,5,6,7,8
0,865,20,24,26,55,5,13,45,7
1,23,1054,28,21,63,1,12,25,8
2,28,36,262,9,72,4,1,14,4
3,33,25,12,240,67,0,3,21,7
4,29,40,34,23,976,5,3,34,9
5,22,15,7,11,47,329,1,12,9
6,0,1,0,0,2,1,1079,149,169
7,5,5,0,0,8,0,117,1144,448
8,4,2,0,1,5,0,143,428,1061


              precision    recall  f1-score   support

           0       0.86      0.82      0.84      1060
           1       0.88      0.85      0.87      1235
           2       0.71      0.61      0.66       430
           3       0.73      0.59      0.65       408
           4       0.75      0.85      0.80      1153
           5       0.95      0.73      0.82       453
           6       0.79      0.77      0.78      1401
           7       0.61      0.66      0.64      1727
           8       0.62      0.65      0.63      1644

    accuracy                           0.74      9511
   macro avg       0.77      0.72      0.74      9511
weighted avg       0.74      0.74      0.74      9511



In [6]:
import pickle

with open('CLF_svm_rbf','wb') as f:
    pickle.dump(clf, f)